In [5]:
import sqlite3
import pandas as pd
import numpy as np


## General Exploration of Data

In [33]:
movie_gross = pd.read_csv('../Data/bom.movie_gross.csv')
movie_gross.head()

#domestic gross hard to read due to large numbers
#make domestic_gross abreviated as 'mil'

,title,studio,domestic_gross,foreign_gross,year
0,Toy Story 3,BV,415000000.0,652000000,2010
1,Alice in Wonderland (2010),BV,334200000.0,691300000,2010
2,Harry Potter and the Deathly Hallows Part 1,WB,296000000.0,664300000,2010
3,Inception,WB,292600000.0,535700000,2010
4,Shrek Forever After,P/DW,238700000.0,513900000,2010


In [4]:
#list of table names
conn= sqlite3.connect("../Data/im.db")
sql_query = """
SELECT name FROM sqlite_master  
WHERE type='table'
;"""

pd.read_sql(sql_query,conn)

,name
0,movie_basics
1,directors
2,known_for
3,movie_akas
4,movie_ratings
5,persons
6,principals
7,writers


In [11]:
#Person table
q = """
SELECT *
FROM persons
"""
persons_pd=pd.read_sql(q, conn)
persons_pd.head()
#missing data: birth and death year, primary professions
#no missing person_id or primary_name

,person_id,primary_name,birth_year,death_year,primary_profession
0,nm0061671,Mary Ellen Bauder,NaN,NaN,"miscellaneous,production_manager,producer"
1,nm0061865,Joseph Bauer,NaN,NaN,"composer,music_department,sound_department"
2,nm0062070,Bruce Baum,NaN,NaN,"miscellaneous,actor,writer"
3,nm0062195,Axel Baumann,NaN,NaN,"camera_department,cinematographer,art_department"
4,nm0062798,Pete Baxter,NaN,NaN,"production_designer,art_department,set_decorator"


In [13]:
#Writers table
q = """
SELECT *
FROM writers
"""
writers_pd=pd.read_sql(q, conn)
writers_pd.head()
writers_pd.info()
#many-to-many table(associative entity)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 255873 entries, 0 to 255872
Data columns (total 2 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   movie_id   255873 non-null  object
 1   person_id  255873 non-null  object
dtypes: object(2)
memory usage: 3.9+ MB


In [15]:
#directors table
q = """
SELECT *
FROM directors
"""
directors_pd=pd.read_sql(q, conn)
directors_pd.head()
#many-to-many table(associative entity)

,movie_id,person_id
0,tt0285252,nm0899854
1,tt0462036,nm1940585
2,tt0835418,nm0151540
3,tt0835418,nm0151540
4,tt0878654,nm0089502


In [16]:
#known_for table
q = """
SELECT *
FROM known_for
"""
known_for_pd=pd.read_sql(q, conn)
known_for_pd.head()
#many-to-many table(associative entity)
#"this/these actor is known for many movies"

,person_id,movie_id
0,nm0061671,tt0837562
1,nm0061671,tt2398241
2,nm0061671,tt0844471
3,nm0061671,tt0118553
4,nm0061865,tt0896534


In [17]:
#principals table
q = """
SELECT *
FROM principals
"""
principals_pd=pd.read_sql(q, conn)
principals_pd.head()
#missing many jobs and characters
#some 'jobs' don't have character roles
#none= no values 

,movie_id,ordering,person_id,category,job,characters
0,tt0111414,1,nm0246005,actor,None,"[""The Man""]"
1,tt0111414,2,nm0398271,director,None,None
2,tt0111414,3,nm3739909,producer,producer,None
3,tt0323808,10,nm0059247,editor,None,None
4,tt0323808,1,nm3579312,actress,None,"[""Beth Boothby""]"


In [24]:
principals_pd[principals_pd['category']=='actor']
#dataframe where the category is equal to actor
#looking to see why the category is 'none'

,movie_id,ordering,person_id,category,job,characters
0,tt0111414,1,nm0246005,actor,None,"[""The Man""]"
5,tt0323808,2,nm2694680,actor,None,"[""Steve Thomson""]"
6,tt0323808,3,nm0574615,actor,None,"[""Sir Lachlan Morrison""]"
14,tt0417610,1,nm0532721,actor,None,"[""Lucio""]"
16,tt0417610,3,nm0069209,actor,None,"[""Dr. Samaniego""]"
...,...,...,...,...,...,...
1028175,tt9681728,9,nm10397910,actor,None,"[""Corpsman""]"
1028176,tt9689618,1,nm10439726,actor,None,None
1028177,tt9689618,2,nm10439727,actor,None,None
1028178,tt9689618,3,nm10439724,actor,None,None


In [29]:
#movi_basics table
q = """
SELECT *
FROM movie_basics
"""
movie_basics_pd=pd.read_sql(q, conn)
movie_basics_pd.head()

,movie_id,primary_title,original_title,start_year,runtime_minutes,genres
0,tt0063540,Sunghursh,Sunghursh,2013,175.0,"Action,Crime,Drama"
1,tt0066787,One Day Before the Rainy Season,Ashad Ka Ek Din,2019,114.0,"Biography,Drama"
2,tt0069049,The Other Side of the Wind,The Other Side of the Wind,2018,122.0,Drama
3,tt0069204,Sabse Bada Sukh,Sabse Bada Sukh,2018,NaN,"Comedy,Drama"
4,tt0100275,The Wandering Soap Opera,La Telenovela Errante,2017,80.0,"Comedy,Drama,Fantasy"


In [30]:
#movie_ratings table
q = """
SELECT *
FROM movie_ratings
"""
movie_ratings_pd=pd.read_sql(q, conn)
movie_ratings_pd.head()

,movie_id,averagerating,numvotes
0,tt10356526,8.3,31
1,tt10384606,8.9,559
2,tt1042974,6.4,20
3,tt1043726,4.2,50352
4,tt1060240,6.5,21


In [31]:
movie_ratings_pd.info()
#no null values

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 73856 entries, 0 to 73855
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   movie_id       73856 non-null  object 
 1   averagerating  73856 non-null  float64
 2   numvotes       73856 non-null  int64  
dtypes: float64(1), int64(1), object(1)
memory usage: 1.7+ MB


In [32]:
#movie_akas table
q = """
SELECT *
FROM movie_akas
"""
movie_akas_pd=pd.read_sql(q, conn)
movie_akas_pd.head()
#multiple of the same movie ID, same movies but in different languages and regions
#where certain movies do best

,movie_id,ordering,title,region,language,types,attributes,is_original_title
0,tt0369610,10,Джурасик свят,BG,bg,None,None,0.0
1,tt0369610,11,Jurashikku warudo,JP,None,imdbDisplay,None,0.0
2,tt0369610,12,Jurassic World: O Mundo dos Dinossauros,BR,None,imdbDisplay,None,0.0
3,tt0369610,13,O Mundo dos Dinossauros,BR,None,None,short title,0.0
4,tt0369610,14,Jurassic World,FR,None,imdbDisplay,None,0.0


In [34]:
reviews = pd.read_table('../Data/rt.reviews.tsv', encoding = 'latin-1')
reviews.head()
#need to use latin-1 in order to get it open
#rating given out of 5(stored as string), might be best to only look at numerator 
#use natural language processing to find missing ratings 

,id,review,rating,fresh,critic,top_critic,publisher,date
0,3,A distinctly gallows take on contemporary fina...,3/5,fresh,PJ Nabarro,0,Patrick Nabarro,"November 10, 2018"
1,3,It's an allegory in search of a meaning that n...,NaN,rotten,Annalee Newitz,0,io9.com,"May 23, 2018"
2,3,... life lived in a bubble in financial dealin...,NaN,fresh,Sean Axmaker,0,Stream on Demand,"January 4, 2018"
3,3,Continuing along a line introduced in last yea...,NaN,fresh,Daniel Kasman,0,MUBI,"November 16, 2017"
4,3,... a perverse twist on neorealism...,NaN,fresh,NaN,0,Cinema Scope,"October 12, 2017"


In [39]:
tmdb_movies = pd.read_csv('../Data/tmdb.movies.csv')
tmdb_movies.head()

,Unnamed: 0,genre_ids,id,original_language,original_title,popularity,release_date,title,vote_average,vote_count
0,0,"[12, 14, 10751]",12444,en,Harry Potter and the Deathly Hallows: Part 1,33.533,2010-11-19,Harry Potter and the Deathly Hallows: Part 1,7.7,10788
1,1,"[14, 12, 16, 10751]",10191,en,How to Train Your Dragon,28.734,2010-03-26,How to Train Your Dragon,7.7,7610
2,2,"[12, 28, 878]",10138,en,Iron Man 2,28.515,2010-05-07,Iron Man 2,6.8,12368
3,3,"[16, 35, 10751]",862,en,Toy Story,28.005,1995-11-22,Toy Story,7.9,10174
4,4,"[28, 878, 12]",27205,en,Inception,27.920,2010-07-16,Inception,8.3,22186


In [43]:
tn_movie_budgets = pd.read_csv('../Data/tn.movie_budgets.csv')
tn_movie_budgets.head()
#best table
#can get gross amounts
#forgeign gross = worldwide-domestic

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross
0,1,"Dec 18, 2009",Avatar,"$425,000,000","$760,507,625","$2,776,345,279"
1,2,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,"$410,600,000","$241,063,875","$1,045,663,875"
2,3,"Jun 7, 2019",Dark Phoenix,"$350,000,000","$42,762,350","$149,762,350"
3,4,"May 1, 2015",Avengers: Age of Ultron,"$330,600,000","$459,005,868","$1,403,013,963"
4,5,"Dec 15, 2017",Star Wars Ep. VIII: The Last Jedi,"$317,000,000","$620,181,382","$1,316,721,747"
